In [ ]:
import pandas as pd
import os
import numpy as np
import re
from collections import defaultdict


def read_gemalen_sheet(filepath):
    df = pd.read_excel(filepath, sheet_name="Gemalen", skiprows=2)
    # Remove first two rows
    df = df.iloc[2:].reset_index(drop=True)
    return df

def clean_and_combine_duplicate_columns(df):
    """
    Converts numeric columns and combines duplicate columns
    (same base name before .1, .2, .3) by summing row-wise.
    """

    # Base names without suffix .1, .2, .3
    base_names = df.columns.str.replace(r"\.\d+$", "", regex=True)

    # Group using transpose (future-proof, no warnings)
    df_combined = (
        df.T
        .groupby(base_names)   # group rows (former columns)
        .sum()                 # sum within each group
        .T                     # transpose back to original shape
    )

    return df_combined

def group_columns_by_kgm(columns):
    groups = defaultdict(list)

    for col in columns:
        match = re.match(r"(KGM\d+)", col)
        if match:
            kgm = match.group(1)
            groups[kgm].append(col)

    return dict(groups)

def aggregate_by_grouped(df, grouped):
    # Initialize new DataFrame with the same index as original
    df_new = pd.DataFrame(index=df.index)
    
    for new_col, old_cols in grouped.items():
        # Sum the old columns row-wise; ignore missing columns
        existing_cols = [col for col in old_cols if col in df.columns]
        df_new[new_col] = df[existing_cols].sum(axis=1)
    
    return df_new

def save_columns_to_csv(df, output_path, stations_ids, suffix="_discharge"):
    os.makedirs(output_path, exist_ok=True)
    
    for col in df.columns:
        if col in stations_ids:
            filename = f"{col}{suffix}.csv"
            filepath = os.path.join(output_path, filename)
            # Save with index
            df[[col]].to_csv(filepath, sep=";")
            print(f"Saved: {filepath}")

def clean_numeric(df):
    """
    Convert all columns of a DataFrame to numeric floats.
    - Handles values like '<0.05' by taking the number and dividing by 2.
    - Non-numeric values are converted to NaN.
    """
    
    def process_value(x):
        if pd.isna(x):
            return np.nan
        if isinstance(x, str) and x.startswith('<'):
            try:
                num = float(x[1:])  # remove '<' and convert
                return num / 2
            except:
                return np.nan
        try:
            return float(x)
        except:
            return np.nan
    
    # Apply to all columns
    df_numeric = df.map(process_value)
    return df_numeric

def clean_and_sort_mps(df, column="MPS.Omschrijving"):
    """
    Cleans date-like strings that contain suffixes like '.1', '.2',
    converts the column to datetime, and returns the dataframe sorted by it.
    """
    df = df.copy()
    
    # Ensure column exists
    if column not in df.columns:
        raise KeyError(f"Column '{column}' not found in dataframe.")
    
    # Remove anything like .1, .2, .33, etc. at the end of the string
    df[column] = df[column].astype(str).str.replace(r"\.\d+$", "", regex=True)
    
    # Convert to datetime (assuming format dd-mm-yyyy)
    df[column] = pd.to_datetime(df[column], format="%d-%m-%Y", errors="coerce")

    # Sort rows by this column
    df = df.sort_values(by=column).reset_index(drop=True)

    # Set column as index
    df = df.set_index(column)

    df  = clean_numeric(df)

    # Drop the rows where all values are NaN with the exception of the index
    df = df.dropna(axis=1, how='all').dropna(axis=0, how='all')

    # Set index as column
    df = df.reset_index()

    counts = df[column].value_counts()

    # Find the dates that appear more than once
    duplicates = counts[counts > 1]

    if not duplicates.empty:
        print("WARNING: Multiple rows with the same datetime found:")
        # Group by datetime and take mean of numeric columns
        df = df.groupby(column).mean().reset_index()
        print(duplicates)
    else:
        print("No duplicate datetimes found.")

    # Set column as index
    df = df.set_index(column)
    
    return df

def rename_columns(df, substance_dict):
    """
    Rename columns in df according to substance_dict.
    Only keeps mappings where the substance has a non-zero name.
    Handles missing columns gracefully.
    
    Parameters:
        df (pd.DataFrame): Input dataframe
        substance_dict (dict): Dictionary mapping new names to existing columns
    
    Returns:
        pd.DataFrame: DataFrame with renamed columns
    """
    # Keep only valid substances (non-zero values)
    valid_substances = {k: v for k, v in substance_dict.items() if v != 0}

    # Build a rename dictionary: only for columns that exist in df
    rename_dict = {v: k for k, v in valid_substances.items() if v in df.columns}

    # Rename columns
    df_renamed = df.rename(columns=rename_dict)

    return df_renamed

def rename_highest_mean_duplicate(df, colname, new_name):
    """
    Rename ONLY one duplicated column with highest mean.
    Handles columns with identical names by tracking occurrence index.
    """
    # Collect the positions of columns with this name
    indices = [i for i, c in enumerate(df.columns) if c == colname]

    if len(indices) < 2:
        return df  # nothing to do

    # Compute means per duplicated occurrence
    means = {}
    for occ, col_idx in enumerate(indices):
        means[occ] = df.iloc[:, col_idx].mean()

    # Which occurrence has the highest mean?
    highest_occ = max(means, key=means.get)

    # Build new column names
    new_columns = df.columns.tolist()

    # Rename only that occurrence
    col_to_rename_idx = indices[highest_occ]
    new_columns[col_to_rename_idx] = new_name

    # Apply back
    df = df.copy()
    df.columns = new_columns
    return df

In [ ]:
file_path = r"P:\ltv-natuur-schelde-slib-waq\ecolmod\02_preprocessing\loads\data waterkwaliteit gemalen Westerschelde.xlsx"

# Your substance dictionary
substance_dict = {
    "DOC": "koolstof organisch",
    "POC1": "Onopgeloste bestandsdelen",
    "NH4": "ammonium",
    "NO3": "som nitraat en nitriet",
    "PON1": "stikstof totaal",
    "PO4": "fosfaat",
    "POP1": "fosfor totaal",
    "Si": 0,
    "Opal": 0,
    "OXY": "zuurstof",
    "Diat": 0,
    "Green": 0
}

# Keep only substances that have meaningful names (not zeros)
valid_substances = {k: v for k, v in substance_dict.items() if v != 0}

output_path = (
    r"P:\ltv-natuur-schelde-slib-waq\ecolmod\02_preprocessing\loads"
)
summary_file = os.path.join(output_path, "summary_all_sheets.csv")

# Create an empty list to collect all rows
summary_rows = []
stations_ids = []

with pd.ExcelFile(file_path) as xls:
    for sheet_name in xls.sheet_names[:]:  # limit to first sheet for now
        print(f"\nProcessing sheet: {sheet_name.split('_')[-1]}")

        df = pd.read_excel(xls, sheet_name=sheet_name, skiprows=5)
        if df.empty:
            print("  (Sheet is empty after skipping rows)")
            continue

        # Transpose and clean
        df = df.transpose()
        df.reset_index(inplace=True)
        df.columns = df.iloc[0]
        df = df[1:].reset_index(drop=True)

        available_cols = df.columns.tolist()

        # Matching columns
        found = [v for v in valid_substances.values() if v in available_cols]
        not_found = [v for v in valid_substances.values() if v not in available_cols]

        print(f"  Found {len(found)} columns: {found}")
        print(f"  Missing {len(not_found)} columns: {not_found}")

        # Build the extraction column list
        extract_cols = ["MPS.Omschrijving"] + found

        # Check MPS exists
        if "MPS.Omschrijving" not in available_cols:
            print("  WARNING: 'MPS.Omschrijving' not found in this sheet.")
            continue

        df_sub = df[extract_cols]

        df_sub = clean_and_sort_mps(df_sub, "MPS.Omschrijving")

        df_sub = rename_columns(df_sub, substance_dict)

        df_sub = rename_highest_mean_duplicate(df_sub, "OXY", "OXY_saturation")

        df_sub.reset_index(inplace=True)
        df_sub.rename(columns={"MPS.Omschrijving": "date"}, inplace=True)
        df_sub.set_index("date", inplace=True)

        # Save the sub dataframe to a csv file
        df_sub.to_csv(os.path.join(output_path,f"{sheet_name.split('_')[-1]}_substance.csv"), sep=";")

        stations_ids.append(sheet_name.split('_')[-1])

        print("The number of rows in the sub dataframe is: ", df_sub.shape[0])
        print("The number of columns in the sub dataframe is: ", df_sub.shape[1])

        # Append summary row
        summary_rows.append({
            "sheet": sheet_name,
            "found_columns": ", ".join(found),
            "missing_columns": ", ".join(not_found),
            "n_found": len(found),
            "n_missing": len(not_found)
        })

# Convert collected rows into a DataFrame
summary_df = pd.DataFrame(summary_rows)

# Save as a single CSV file using semicolon as separator
summary_df.to_csv(summary_file, index=False, sep=";")

print(f"\n✔ Combined summary saved to:\n{summary_file}")



Processing sheet: KGM134
  Found 8 columns: ['koolstof organisch', 'Onopgeloste bestandsdelen', 'ammonium', 'som nitraat en nitriet', 'stikstof totaal', 'fosfaat', 'fosfor totaal', 'zuurstof']
  Missing 0 columns: []
No duplicate datetimes found.
The number of rows in the sub dataframe is:  12
The number of columns in the sub dataframe is:  9

Processing sheet: KGM135
  Found 7 columns: ['Onopgeloste bestandsdelen', 'ammonium', 'som nitraat en nitriet', 'stikstof totaal', 'fosfaat', 'fosfor totaal', 'zuurstof']
  Missing 1 columns: ['koolstof organisch']
No duplicate datetimes found.
The number of rows in the sub dataframe is:  12
The number of columns in the sub dataframe is:  8

Processing sheet: KGM136
  Found 8 columns: ['koolstof organisch', 'Onopgeloste bestandsdelen', 'ammonium', 'som nitraat en nitriet', 'stikstof totaal', 'fosfaat', 'fosfor totaal', 'zuurstof']
  Missing 0 columns: []
No duplicate datetimes found.
The number of rows in the sub dataframe is:  12
The number of 

In [ ]:
filepath = r"P:\ltv-natuur-schelde-slib-waq\ecolmod\02_preprocessing\loads\Debieten_gemalen_stuwen_westerschelde_2018.xlsx"

df_gemalen = read_gemalen_sheet(filepath)
df_gemalen.rename(columns={"Unnamed: 0": "date"}, inplace=True)
df_gemalen.set_index("date", inplace=True)
df_gemalen  = clean_numeric(df_gemalen)
cols = df_gemalen.columns  # or your list

grouped = group_columns_by_kgm(cols)
grouped

df_clean = clean_and_combine_duplicate_columns(df_gemalen)
df_aggregated = aggregate_by_grouped(df_gemalen, grouped)
save_columns_to_csv(df_aggregated, output_path, stations_ids)

In [ ]:
# Read gemalen sheet
from dwq_utilities import read_waarde_sheet
import os
import pandas as pd
data_dir = r"P:\ltv-natuur-schelde-slib-waq\ecolmod\02_preprocessing\loads"
waarde_path = os.path.join(data_dir, "Concentraties_en_vrachten_in_de_waterlijn.xlsx")

df = pd.read_excel(waarde_path, sheet_name="effluent rwzi Waarde", skiprows=9)
df = df.iloc[3:].reset_index(drop=True)
# Remove last 10 rows
df = df.iloc[:-11].reset_index(drop=True)
# Remove columns Unnamed
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

# Change column Parameter to datetime using var time_col and  new_time_col
time_col = "Parameter"
new_time_col = "datetime"
df[new_time_col] = pd.to_datetime(df[time_col])
df.drop(columns=[time_col], inplace=True)
# Set time_col as index
df.set_index(new_time_col, inplace=True)

# Convert all columns to numeric
df = df.apply(pd.to_numeric, errors='coerce')
df


substance_dict = {
    "DOC": "CZV",        # Chemisch zuurstofverbruik
    "POC1": "OB",        # Onopgeloste bestandsdelen
    "NH4": "NH4",        # Ammonium
    "NO3": "NO3",        # Nitraat
    "PON1": "Ntot",      # Stikstof totaal
    "PO4": "PO4",        # Fosfaat
    "POP1": "Ptot",      # Fosfor totaal
    "Si": 0,
    "Opal": 0,
    "OXY": "TZV",        # Totaal zuurstofverbruik
    "Diat": 0,
    "Green": 0,
    "Q": "Q"
}

# Keep only substances that have meaningful names (not zeros)
valid_substances = {k: v for k, v in substance_dict.items() if v != 0}
valid_substances

# Keep only valid substances and rename with keys of valid_substances
# Filter dataframe columns to keep only those present in valid_substances values
df_filtered = df[list(valid_substances.values())].copy()

# Rename columns from new names → original keys (model names)
rename_dict = {v: k for k, v in valid_substances.items()}
df_filtered.rename(columns=rename_dict, inplace=True)

df_filtered
# Save to csv
df_filtered.to_csv(os.path.join(data_dir, "waarde_effluent_rwzi.csv"), sep=";")